In [14]:
import pandas as pd

def read_csv_column(csv_path, column_name):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    return df[column_name].tolist()

def read_csv_with_weight(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    # 转发数,评论数,点赞数
    comments = df['评论数'].tolist()
    reposts = df['转发数'].tolist()
    likes = df['点赞数'].tolist()
    weibos = df['微博正文'].tolist()
    results = []

    for row in range(len(weibos)):
        if comments[row] + reposts[row] + likes[row] > 10:
            results.append(weibos[row])

    return results


In [15]:
contents = read_csv_column('./trump/preprocessed_v2.csv', '微博正文')

new_list = []

for content in contents:
    content = content.split()
    new_list.append(content)


In [16]:
from gensim.corpora import Dictionary

'''
Create a dictionary from 'processed_docs' containing the number of times a word appears
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''

dictionary = Dictionary(new_list)

dictionary.filter_extremes(no_below=5, no_above=0.85, keep_n=10000)

'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in new_list]

document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0],
                                                     dictionary[bow_doc_x[i][0]],
                                                     bow_doc_x[i][1]))

Word 27 ("说话") appears 2 time.
Word 50 ("时间") appears 2 time.
Word 122 ("关注") appears 1 time.
Word 198 ("30日") appears 1 time.
Word 199 ("BBC") appears 1 time.
Word 200 ("专家") appears 1 time.
Word 201 ("人身") appears 1 time.
Word 202 ("可笑") appears 1 time.
Word 203 ("回应") appears 1 time.
Word 204 ("大会") appears 1 time.
Word 205 ("对此") appears 1 time.
Word 206 ("对话") appears 1 time.
Word 207 ("少女") appears 1 time.
Word 208 ("接受") appears 1 time.
Word 209 ("推特") appears 1 time.
Word 210 ("攻击") appears 1 time.
Word 211 ("死亡") appears 1 time.
Word 212 ("气候") appears 1 time.
Word 213 ("浪费") appears 2 time.
Word 214 ("焦点") appears 1 time.
Word 215 ("狠狠") appears 1 time.
Word 216 ("环保") appears 1 time.
Word 217 ("瑞典") appears 1 time.
Word 218 ("生活") appears 1 time.
Word 219 ("科学家") appears 1 time.
Word 220 ("联合国") appears 1 time.
Word 221 ("讽刺") appears 1 time.
Word 222 ("采访") appears 1 time.


In [17]:
from gensim.models import LdaMulticore

# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus,
#                                    num_topics = 10,
#                                    id2word = dictionary,
#                                    passes = 50)

# LDA multicore
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model = LdaMulticore(bow_corpus,
                               num_topics = 100,
                               id2word = dictionary,
                               passes = 10,
                               workers = 4)

'''
For each topic, we will explore the words occurring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

KeyboardInterrupt: 

In [ ]:
trump1_content = read_csv_with_weight('./trump/trump_1.csv')

item = ''
for i in range(len(trump1_content)):
    item = item + trump1_content[i]
unseen_document = item.split()

bow_vector = dictionary.doc2bow(unseen_document)

print("4月\n")
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 20)))